# SurrealEngine v0.7.0 Features

This notebook demonstrates the major features introduced in SurrealEngine v0.7.0:

1. **Vector & Full-Text Search** - HNSW indexes and BM25 search
2. **Server-Side Functions** - `@surreal_func` decorator
3. **Event Triggers** - `Event` class and `DEFINE EVENT`
4. **Reactive Queries** - `ReactiveQuerySet` and `.watch()`
5. **Embedded Documents** - `EmbeddedDocument` with change tracking
6. **Polyglot API** - Seamless sync/async support (demo in `polyglot_api.ipynb`)
7. **Zero-Copy Accelerator** - Rust-based performance (demo in `zero_copy_benchmark.ipynb`)

In [1]:
# Setup
import asyncio
from surrealengine import (
    Document, EmbeddedDocument, Event,
    StringField, IntField, FloatField, ListField, 
    VectorField, EmbeddedField, DictField,
    create_connection, surreal_func, functions
)


In [2]:
# Connect to SurrealDB
conn = create_connection(
    url="ws://db:8000/rpc",
    username="root",
    password="root",
    namespace="demo",
    database="v070_features",
    make_default=True
)
await conn.connect()
print("Connected to SurrealDB!")

Connected to SurrealDB!


## 1. Vector & Full-Text Search

v0.7.0 adds support for defining HNSW vector indexes and Full-Text Search indexes in your defined schema.

In [3]:
class Article(Document):
    title = StringField()
    content = StringField()
    embedding = VectorField(dimension=4) # Simple 4D vector for demo
    
    class Meta:
        collection = "articles"
        indexes = [
            # Full Text Search Index
            {
                "name": "idx_content",
                "fields": ["content"],
                "search": True,  # Enables FTS
                "bm25": True,    # Use BM25 algorithm
                "highlights": True # Enable highlighting
            },
            # Vector Search Index (HNSW)
            {
                "name": "idx_embedding",
                "fields": ["embedding"],
                "m": 16,
                "efc": 100,
                "dimension": 4,
                "dist": "euclidean" # or 'cosine', 'manhattan', 'hamming'
            }
        ]

# Create table and indexes
await Article.create_table()

## 2. Server-Side Functions

You can now define SurrealDB stored functions directly from Python using the `@surreal_func` decorator.

In [3]:
@surreal_func("fn::greet")
def greet_person(name: str):
    """
    RETURN "Hello, " + $name + "!";
    """
    pass

@surreal_func("fn::calculate_score")
def calculate_score(base: int, multiplier: float):
    """
    RETURN $base * $multiplier;
    """
    pass

# Deploy functions to the database
statements = functions.generate_function_statements()
print("Deploying functions:")
for stmt in statements:
    print(stmt)
    await conn.client.query(stmt)

Deploying functions:
DEFINE FUNCTION fn::greet($name: string) {
    RETURN "Hello, " + $name + "!";
};
DEFINE FUNCTION fn::calculate_score($base: int, $multiplier: float) {
    RETURN $base * $multiplier;
};


## 3. Event Triggers

Define `DEFINE EVENT` logic directly on your documents using the `Event` class in `Meta.events`.

In [5]:
class AuditLog(Document):
    action = StringField()
    record_id = StringField()
    timestamp = StringField()
    class Meta:
        collection = "audit_log"

class SensitiveDoc(Document):
    secret = StringField()
    
    class Meta:
        collection = "sensitive"
        events = [
            Event(
                name="log_changes",
                when="AFTER",  # or BEFORE
                then="""
                CREATE audit_log SET 
                    action = $event, 
                    record_id = $after.id, 
                    timestamp = time::now()
                """
            )
        ]

await AuditLog.create_table()
await SensitiveDoc.create_table()

## 4. Reactive Queries

`ReactiveQuerySet` allows you to subscribe to live query updates and maintain a real-time list of records.

In [ ]:
class LiveItem(Document):
    name = StringField()
    score = IntField()
    class Meta:
        collection = "live_items"

await LiveItem.create_table()

# Create a reactive collection (Top 5 items by score)
# Note: In a real app, you would keep this running. 
# Here we simulate it.
reactive_items = await LiveItem.objects.order_by("score", "DESC").limit(5).reactive()

print(f"Initial items: {len(reactive_items.items)}")

# Create an item and see if we catch it (requires running loop, difficult in static notebook execution, demonstrates API)
# async for changes in reactive_items.watch():
#     print("Change detected!")

## 5. Embedded Documents & Change Tracking

v0.7.0 brings full support for embedded documents with automatic change tracking for nested updates.

In [9]:
class Address(EmbeddedDocument):
    street = StringField()
    city = StringField()
    zip_code = IntField()

class Person(Document):
    name = StringField()
    address = EmbeddedField(Address)
    tags = ListField(StringField())
    
    class Meta:
        collection = "person"



In [10]:
# Create with nested data
p = Person(
    name="John Doe",
    address=Address(street="123 Main St", city="New York", zip_code=10001),
    tags=["new", "customer"]
)
await p.save()
print(f"Saved: {p.name} in {p.address.city}")

Saved: John Doe in New York


In [11]:
# Demonstrating Nested Change Tracking
# We can modify nested fields directly and .save() will detect it

p.address.city = "San Francisco"  # Modify embedded field
p.tags.append("moved")            # Modify list in-place

await p.save()
print("Updated person! Check database to verify persistence.")

# Verify
await p.refresh()
print(f"City: {p.address.city}")
print(f"Tags: {p.tags}")

Updated person! Check database to verify persistence.
City: San Francisco
Tags: ['new', 'customer', 'moved']


## Cleanup

In [ ]:
await conn.disconnect()
print("Disconnected.")